In [1]:
import pymc as pm
import pandas as pd
import numpy as np
import arviz as az
from pymc.math import dot
import pytensor.tensor as pt

%load_ext lab_black

# 13. Multiple Regression (Demo)

Adapted from [fat2d.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/fat1.odc) and [fatmulti.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/fatmulti.odc) (they appear to be identical). This is the same dataset as [](./Unit7/demo-regression.ipynb), available for download [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/fat.tsv).

In [2]:
data = pd.read_csv("../data/fat.tsv", sep="\t")

y = data["y"].to_numpy(copy=True)
X = data.iloc[:, 1:].to_numpy(copy=True)

# add intercept
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
n, p = X_aug.shape

# Zellner's g
g = p**2

n, p, g

(252, 15, 225)

In [3]:
X_aug.shape

(252, 15)

In [4]:
mu_beta = np.zeros(p)

In [5]:
with pm.Model() as m2d:
    tau = pm.Gamma("tau", 0.01, 0.01)
    variance = pm.Deterministic("variance", 1 / tau)

    tau_matrix = pt.fill(pt.zeros((15, 15)), tau)
    tau_beta = tau_matrix / g * dot(X_aug.T, X_aug)
    beta = pm.MvNormal("beta", mu_beta, tau=tau_beta)

    mu = dot(X_aug, beta)
    pm.Normal("likelihood", mu=mu, tau=tau, observed=y)

    # Bayesian R2 from fat2d.odc
    sse = (n - p) * variance
    cy = y - y.mean()
    sst = dot(cy, cy)
    br2 = pm.Deterministic("br2", 1 - sse / sst)
    br2_adj = pm.Deterministic("br2_adj", 1 - (n - 1) * variance / sst)

    trace = pm.sample(1000)
    ppc = pm.sample_posterior_predictive(trace)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, beta]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 303 seconds.
Sampling: [likelihood]


In [6]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],-15.194,16.247,-47.402,15.271,0.378,0.270,1848.0,2197.0,1.0
beta[1],0.056,0.030,-0.000,0.117,0.001,0.000,2840.0,2779.0,1.0
beta[2],-0.081,0.051,-0.180,0.020,0.001,0.001,1904.0,2276.0,1.0
beta[3],-0.051,0.105,-0.252,0.158,0.002,0.002,2354.0,2564.0,1.0
beta[4],0.067,0.282,-0.477,0.629,0.006,0.004,2522.0,2951.0,1.0
beta[5],-0.445,0.223,-0.887,-0.015,0.004,0.003,3368.0,2648.0,1.0
beta[6],-0.030,0.102,-0.238,0.167,0.002,0.001,2832.0,2976.0,1.0
beta[7],0.875,0.088,0.704,1.045,0.002,0.001,2974.0,2991.0,1.0
beta[8],-0.205,0.139,-0.477,0.066,0.003,0.002,2613.0,2747.0,1.0
beta[9],0.227,0.140,-0.052,0.494,0.003,0.002,3136.0,2892.0,1.0


In [7]:
y_pred = ppc.posterior_predictive.stack(sample=("chain", "draw"))["likelihood"].values.T
az.r2_score(y, y_pred)

r2        0.653819
r2_std    0.024084
dtype: float64

- https://arxiv.org/abs/1702.01201

- https://towardsdatascience.com/linear-regression-model-selection-through-zellners-g--prior-da5f74635a03

- https://en.wikipedia.org/wiki/G-prior\

Original paper:

Zellner, A. (1986). "On Assessing Prior Distributions and Bayesian Regression Analysis with g Prior Distributions". In Goel, P.; Zellner, A. (eds.). Bayesian Inference and Decision Techniques: Essays in Honor of Bruno de Finetti. Studies in Bayesian Econometrics and Statistics. Vol. 6. New York: Elsevier. pp. 233 243. ISBN 978-0-444-87712-3.

In [8]:
%load_ext watermark
%watermark -n -u -v -iv -p pytensor

Last updated: Wed Mar 22 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

pytensor: 2.10.1

pandas  : 1.5.3
arviz   : 0.14.0
numpy   : 1.24.2
pytensor: 2.10.1
pymc    : 5.1.2

